In [38]:
!pip install sentencepiece 

In [39]:
!pip install transformers

In [40]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
import nltk

import re

In [41]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
comments = pd.read_csv('/content/drive/MyDrive/IMDB_Dataset/IMDB Dataset.csv')

In [44]:
le = LabelEncoder()
#comments['sentiment_encoded'] = le.fit_transform(comments['sentiment'])

In [77]:
comments.shape

(50000, 2)

In [45]:
df_red = comments[['review', 'sentiment']].iloc[:300]
df_red['sen_enc'] = LabelEncoder().fit_transform(df_red['sentiment'])

In [46]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

cuda


In [47]:
#DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, 
                                                    ppb.DistilBertTokenizer, 
                                                    'distilbert-base-uncased')

# # AlBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.AlbertModel, 
#                                                     ppb.AlbertTokenizerFast,
#                                                     'albert-base-v1')

# BERT
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, 
#                                                     ppb.BertTokenizer, 
#                                                     'bert-base-uncased')

# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, 
#                                                      ppb.BertTokenizer, 
#                                                      'bert-base-cased')


# Load pretrained model/tokenizer

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
df_red.columns

Index(['review', 'sentiment', 'sen_enc'], dtype='object')

In [49]:
# a = tokenizer_class.from_pretrained('distilbert-base-uncased')
# type(a)

In [50]:
df_red['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [51]:
tokenized = df_red['review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (563 > 512). Running this sequence through the model will result in indexing errors


In [52]:
len(tokenized[101])

969

In [53]:
type(tokenized)

pandas.core.series.Series

In [54]:
tokenized512 = tokenized.apply(lambda x: x[:500])

In [55]:
len(tokenized512[101])

500

In [56]:
# df512a = df_red['review'].apply(lambda x: x.split())
# len(df512a[101])

In [57]:
# print(df512a[101])

In [58]:
# df512 = df_red['review'].apply(lambda x: x.split()).apply(lambda x: x[:512])
# len(df512[101])

In [59]:
# df512_joint = df512.apply(lambda x: ' '.join(x))
# len(df512_joint[101])

In [60]:
# df512_joint[101]

In [61]:
# len(df512_joint[101].split())

In [62]:
#tokenized = df512_joint.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [63]:
max_len = 0
for i in tokenized512.values:
    if len(i) > max_len:
        max_len = len(i)
        #print(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized512.values])

In [64]:
max_len

500

In [65]:
np.array(padded).shape

(300, 500)

In [66]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(300, 500)

In [67]:
type(tokenizer)

transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer

In [68]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)


# Получаем выход модели
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [69]:
input_ids.shape

torch.Size([300, 500])

In [70]:
features = last_hidden_states[0][:,0,:].numpy()

In [71]:
labels = df_red['sen_enc'].values

In [72]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [73]:
lr_clf = LogisticRegression(C=4)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
lr_clf.score(test_features, test_labels)

0.8666666666666667

In [75]:
# ppb.BertModel, ppb.BertTokenizer,  'bert-base-cased': 0.6933333333333334  df_red = comments[['review', 'sentiment']].iloc[:300]

In [75]:
# ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased': 0.8666666666666667 df_red = comments[['review', 'sentiment']].iloc[:300]